In [35]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from keras.utils import to_categorical
import tensorflow as tf
from keras.optimizers import Adam
import numpy as np
import cv2

import glob
import os


In [36]:
size = 180

input = Input(shape = (size, size, 1))

conv1 = Conv2D(10, 4, activation='relu')(input)
dropout1 = Dropout(0.5)(conv1)

batch1 = tf.keras.layers.BatchNormalization(
    axis=-1,
    momentum=0.99,
    epsilon=0.001,
    center=True,
    scale=True,
    beta_initializer="zeros",
    gamma_initializer="ones",
    moving_mean_initializer="zeros",
    moving_variance_initializer="ones",
    beta_regularizer=None,
    gamma_regularizer=None,
    beta_constraint=None,
    gamma_constraint=None,
    synchronized=False,
)(dropout1)

pool1 = MaxPool2D(pool_size=(3, 3))(batch1)

conv2 = Conv2D(20, 4, activation='relu')(pool1)
dropout2 = Dropout(0.5)(conv2)
batch2 = tf.keras.layers.BatchNormalization(
    axis=-1,
    momentum=0.99,
    epsilon=0.001,
    center=True,
    scale=True,
    beta_initializer="zeros",
    gamma_initializer="ones",
    moving_mean_initializer="zeros",
    moving_variance_initializer="ones",
    beta_regularizer=None,
    gamma_regularizer=None,
    beta_constraint=None,
    gamma_constraint=None,
    synchronized=False,
)(dropout2)
pool2 = MaxPool2D(pool_size=(3, 3))(dropout2)

flat = Flatten()(pool2)
hidden = Dense(128, activation='relu')(flat)
output = Dense(4, activation='softmax')(hidden)
model = Model(inputs=input, outputs=output)

In [37]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [38]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 180, 180, 1)]     0         
                                                                 
 conv2d_6 (Conv2D)           (None, 177, 177, 10)      170       
                                                                 
 dropout_6 (Dropout)         (None, 177, 177, 10)      0         
                                                                 
 batch_normalization_6 (Batc  (None, 177, 177, 10)     40        
 hNormalization)                                                 
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 59, 59, 10)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 56, 56, 20)        3220

## Train Datasets

In [39]:
datasets = "datasets_split/train/"

N = 1800

x_train = np.zeros((N, size, size, 1), 'float')
y_train = np.zeros((N), 'float')

count = 0

class_id = sorted(os.listdir(datasets))

for i, class_ in enumerate(class_id):
    datas = sorted(os.listdir(os.path.join(datasets, class_)))
    
    for data in datas:
        im = cv2.imread(os.path.join(datasets, class_, data), cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im, (size, size))
        x_train[count, :, :, 0] = im / 255.
        y_train[count] = i
        count += 1

y_train = to_categorical(y_train)

### Train Model

In [40]:
h = model.fit(x_train, y_train, epochs=50, batch_size=64, verbose=1)

Epoch 1/50
29/29 [==============================] - 21s 697ms/step - loss: 1.8738 - accuracy: 0.4356
Epoch 2/50
29/29 [==============================] - 22s 742ms/step - loss: 1.1976 - accuracy: 0.4972
Epoch 3/50
29/29 [==============================] - 21s 722ms/step - loss: 1.0190 - accuracy: 0.5567
Epoch 4/50
29/29 [==============================] - 20s 691ms/step - loss: 0.9543 - accuracy: 0.5872
Epoch 5/50
29/29 [==============================] - 20s 703ms/step - loss: 0.8594 - accuracy: 0.6422
Epoch 6/50
29/29 [==============================] - 21s 708ms/step - loss: 0.8142 - accuracy: 0.6467
Epoch 7/50
29/29 [==============================] - 21s 710ms/step - loss: 0.7762 - accuracy: 0.6711
Epoch 8/50
29/29 [==============================] - 21s 721ms/step - loss: 0.7116 - accuracy: 0.7161
Epoch 9/50
29/29 [==============================] - 22s 766ms/step - loss: 0.6826 - accuracy: 0.7261
Epoch 10/50
29/29 [==============================] - 22s 760ms/step - loss: 0.6665 - accura

In [41]:
# save model
model.save("datasets_model.h5")

## Test Datasets

In [42]:
datasets = "datasets_split/test/"

N = 500

x_test = np.zeros((N, size, size, 1), 'float')
y_test = np.zeros((N), 'float')

count = 0

class_id = sorted(os.listdir(datasets))

for i, class_ in enumerate(class_id):
    datas = sorted(os.listdir(os.path.join(datasets, class_)))
    
    for data in datas:
        im = cv2.imread(os.path.join(datasets, class_, data), cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im, (size, size))
        x_test[count, :, :, 0] = im / 255.
        y_test[count] = i
        count += 1

y_test = to_categorical(y_test)

## Validation Datasets

In [43]:
datasets = "datasets_split/val/"

N = 500

x_val = np.zeros((N, size, size, 1), 'float')
y_val = np.zeros((N), 'float')

count = 0

class_id = sorted(os.listdir(datasets))

for i, class_ in enumerate(class_id):
    datas = sorted(os.listdir(os.path.join(datasets, class_)))
    
    for data in datas:
        im = cv2.imread(os.path.join(datasets, class_, data), cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im, (size, size))
        x_val[count, :, :, 0] = im / 255.
        y_val[count] = i
        count += 1

y_val = to_categorical(y_val)

### Train Model

In [44]:
h = model.fit(x_train, y_train, epochs=50, batch_size=64, verbose=1, validation_data=(x_val, y_val))

Epoch 1/50
29/29 [==============================] - 23s 786ms/step - loss: 0.0263 - accuracy: 0.9972 - val_loss: 0.9260 - val_accuracy: 0.6040
Epoch 2/50
29/29 [==============================] - 23s 787ms/step - loss: 0.0215 - accuracy: 0.9983 - val_loss: 0.9385 - val_accuracy: 0.6060
Epoch 3/50
29/29 [==============================] - 23s 780ms/step - loss: 0.0251 - accuracy: 0.9967 - val_loss: 0.9103 - val_accuracy: 0.6200
Epoch 4/50
29/29 [==============================] - 24s 816ms/step - loss: 0.0201 - accuracy: 0.9983 - val_loss: 0.9395 - val_accuracy: 0.6080
Epoch 5/50
29/29 [==============================] - 24s 834ms/step - loss: 0.0266 - accuracy: 0.9967 - val_loss: 0.9712 - val_accuracy: 0.5920
Epoch 6/50
29/29 [==============================] - 25s 869ms/step - loss: 0.0167 - accuracy: 0.9994 - val_loss: 0.9463 - val_accuracy: 0.6040
Epoch 7/50
29/29 [==============================] - 26s 895ms/step - loss: 0.0178 - accuracy: 0.9989 - val_loss: 0.9464 - val_accuracy: 0.6000

In [45]:
y_predic = model.predict(x_test)

prediction = y_predic.argmax(axis=1)


16/16 [==============================] - 1s 64ms/step


In [46]:
classifications = ['B', 'D', 'R', 'S']

data_test = 'datasets_split/test/'

images_test = []

for folder in os.listdir(data_test):
    folders = os.path.join(data_test, folder)
    
    if os.path.isdir(folders):
        
        for data in os.listdir(folders):
            
            if data.endswith('.jpg'):
                data_path = os.path.join(folders, data)
                images_test.append(data_path)

In [47]:
with open('result.txt', 'w') as f:
    
    for i in range(len(images_test)):
        image_name = os.path.basename(images_test[i])
        classification = classifications[prediction[i]]
        f.write("{}::{}".format(
            image_name, classification))

In [48]:
loss, accuracy = model.evaluate(x_test, y_test)

print(f'Total loss: {loss}')
print(f'Total accuracy: {accuracy}')

16/16 [==============================] - 1s 73ms/step - loss: 0.4666 - accuracy: 0.8140
Total loss: 0.46659308671951294
Total accuracy: 0.8140000104904175
